In [8]:
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image
from crossword_algorithm import find_words
import random

In [9]:
#while True:
    # ret, frame3 = video.read()

    # frame4 = cv2.cvtColor(frame3, cv2.COLOR_BGR2GRAY)
    # #frame = cv2.blur(frame4, (100,1))

    # frame = cv2.adaptiveThreshold(frame4, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 7)

    # cv2.imshow("vid", frame)

    # if cv2.waitKey(4) & 0xFF == ord('q'):
    #     break

# video.release()
# cv2.destroyAllWindows()

In [10]:
image2 = cv2.imread("image_samples/word_search_school.png")
image = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
#ret, image = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY_INV)
image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 33, 25)

image = cv2.blur(image, (2,2))

contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

area = 0

for cnt in contours:
    area += cv2.contourArea(cnt)

area = area/len(contours)

letter_grid_coordinates = dict()
y_top, y_bottom = -1,-1

for i in range(len(contours)):
    #cv2.drawContours(image2, cnt, -1, (255,0,0), 7 )
    if cv2.contourArea(contours[i]) < 0.1*area:
        continue
    x, y, w, h = cv2.boundingRect(contours[i])
    #cv2.rectangle(image2, (x - int(w*(0.45)), y - int(h*(0.45))), (x + w + int(w*(0.45)), y + h + int(h*(0.45))), (0, 255, 0), 2)

    pushed = False

    for key in letter_grid_coordinates.keys():
        if y + w//2 in range(key[0], key[1]):
            y_top, y_bottom = y, y+h
            letter_grid_coordinates[key].append((x - int((w)*(0.25)), y - int(h*(0.25)), x + w + int(w*(0.25)), y + h + int(h*(0.25))))
            pushed = True

    if not pushed:
        letter_grid_coordinates[(y, y+w)] = [(x - int(w*(0.25)), y - int(h*(0.25)), x + w + int(w*(0.25)), y + h + int(h*(0.25)))]
    
    #cv2.rectangle(image2, (x - int(w*(0.45)), y - int(h*(0.45))), (x + w + int(w*(0.45)), y + h + int(h*(0.45))), (0, 255, 0), 2)
    

In [11]:
# video = cv2.VideoCapture(0)

# while True:
#     r, image2 = video.read()
#     image = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
#     #ret, image = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY_INV)
#     image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 33, 25)

#     image = cv2.blur(image, (2,2))

#     contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     area = 0

#     for cnt in contours:
#         area += cv2.contourArea(cnt)

#     area = area/len(contours)

#     for i in range(len(contours)):
#         #cv2.drawContours(image2, cnt, -1, (255,0,0), 7 )
#         if cv2.contourArea(contours[i]) < 0.1*area:
#             continue
#         x, y, w, h = cv2.boundingRect(contours[i])
#         cv2.rectangle(image2, (x - int(w*(0.45)), y - int(h*(0.45))), (x + w + int(w*(0.45)), y + h + int(h*(0.45))), (0, 255, 0), 2)

#     cv2.imshow("hsf", image2)
#     cv2.imshow("hsf2", image)

#     cv2.waitKey(80)

In [12]:
model = tf.keras.models.load_model("font_identifier.keras")
letter_width = None

In [ ]:
cv2.imshow("hsf", image2)
cv2.imshow("hsf2", image)
cv2.waitKey(1000)
cv2.destroyAllWindows()

crossword = list()

for i in letter_grid_coordinates.keys():
    letter_grid_coordinates[i] = sorted(letter_grid_coordinates[i], key=lambda x: x[0])
    #print(letter_grid_coordinates[i])

for i in reversed(letter_grid_coordinates.values()):
    x = list()
    for j in i:
        x1, y1, x2, y2 = j
        if not letter_width:
            letter_width = y2 - y1
        img = image[y1:y2, x1:x2]
        img = Image.fromarray(img)
        img.thumbnail((28, 28), Image.Resampling.LANCZOS)

        pillow_image = Image.new("L", (28, 28), 0)
        pillow_image.paste(img, ((28 - img.size[0]) // 2, (28 - img.size[1]) // 2))

        img = np.array(pillow_image).reshape((1,28,28,1))
        img = img/255.0
        
        prediction = model.predict(img)
        x.append(chr(np.argmax(prediction) + 65))


    crossword.append(x)

# j = list(letter_grid_coordinates.values())[8][17]

# x1, y1, x2, y2 = j
# img = image[y1:y2, x1:x2]
# img = cv2.resize(img, (28, 28))
# cv2.imshow(str(x1), img)
# img = img.reshape((-1,28,28,1))
# img = img/255.0
# prediction = model.predict(img)
# print(chr(np.argmax(prediction) + 65))

# for i in letter_grid_coordinates.keys():
#     print(len(letter_grid_coordinates[i]) , "elements in", i)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "conv2d" is incompatible with the layer: expected axis -1 of input shape to have value 1, but received input with shape (20, 1, 28, 28)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(20, 1, 28, 28, 1), dtype=float32)
  • training=False
  • mask=None
  • kwargs=<class 'inspect._empty'>

In [ ]:
#word_position = find_words(crossword, ['GARDEN', 'SUMMER', 'SUNSHINE', 'SWIM', 'BOAT', 'CAMP', 'HIKE', 'PLAY', 'BEACH', 'JULY', 'AUGUST', 'PARK', 'PICNIC', 'POPSICLE', 'ICECREAM', 'SHORTS', 'TRAVEL', 'DRESS', 'VACATION', 'SEASON'])


word_position = find_words(crossword, [
    'DRAMA', 'HISTORY', 'NUMBERS', 'SCIENCE', 'ART',
    'ELEMENTARY', 'HOMEWORK', 'PENCIL', 'SOCIALSTUDIES',
    'BACKPACK', 'ENGLISH', 'LANGUAGEARTS', 'PHYSICALEDUCATION',
    'SPELLING', 'BOOKS', 'FRIENDS', 'LEARN', 'READING',
    'STUDENTS', 'CLASSROOM', 'GEOGRAPHY', 'LIBRARY',
    'RECESS', 'SUBJECTS', 'CRAYONS', 'GRADES',
    'MATH', 'SCHOOL', 'TEACHER', 'DESK', 'HEALTH',
    'MUSIC', 'SCISSORS', 'WRITING'
])

image2_cpy = image2.copy()

for x in word_position:
    start_word_pos, end_word_pos = x
    k = list(reversed(letter_grid_coordinates.values()))
    start = k[start_word_pos[0]][start_word_pos[1]]
    end = k[end_word_pos[0]][end_word_pos[1]]

    color = random.sample(range(150, 200), 3)

    #result = cv2.line(result, ((start[0] + start[2]) // 2 - 10, (start[1] + start[3]) // 2 - 10) , ((end[0] + end[2]) // 2 + 10, (end[1] + end[3]) // 2 + 10) , color, 1)
    result = cv2.line(image2, ((start[0] + start[2]) // 2 - 2, (start[1] + start[3]) // 2 - 2) , ((end[0] + end[2]) // 2 + 2, (end[1] + end[3]) // 2 + 2) , color, letter_width)
    
alpha = 0.3
result = cv2.addWeighted(result, alpha, image2_cpy, 1 - alpha, 0)

cv2.imshow("solved", result)
cv2.waitKey(10)


-1